<a href="https://colab.research.google.com/github/zxcv1230174/colorectal_histology/blob/main/CH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, callbacks
import tensorflow_hub as hub
import pandas as pd

# 載入資料

In [ ]:
# 將訓練資料拆分成7:3，分別是訓練與測試集，並且取得Info
[train_ds, valid_ds, test_ds], info = tfds.load('colorectal_histology',
                            split=['train[:70%]','train[70%:85%]', 'train[85%:]'],
                            shuffle_files=True,
                            with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/colorectal_histology/2.0.0.incompleteMPMOIR/colorectal_histology-train.tfr…

Dataset colorectal_histology downloaded and prepared to /root/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.


In [ ]:
# 參數設定
#input_shapes = info.features[info.supervised_keys[0]].shape
input_shapes = (224,224,3)
img_size = input_shapes[:2]
batch_size = 32
epoch = 1

In [ ]:
# 預處理函數
def preprocess_data(data):
    image = data['image']
    label = data['label']
    # 將圖片大小調整為指定大小,並正規化像素值至 [0, 1]
    image = tf.image.resize(image, img_size) / 255.0
    return image, label

# 將預處理函數應用到數據集
train_data = train_ds.map(preprocess_data).batch(batch_size)
valid_data = valid_ds.map(preprocess_data).batch(batch_size)
test_data = test_ds.map(preprocess_data).batch(batch_size)

# 定義模型

In [ ]:
# 建立 VGG16 模型
VGG16_model = tf.keras.applications.VGG16(input_shape=input_shapes, include_top=False, weights='imagenet')
# 凍結 VGG16 的權重
VGG16_model.trainable = False

model_4 = models.Sequential()
model_4.add(VGG16_model)
model_4.add(layers.Flatten())
model_4.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_4.summary()
# 編譯模型
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

58889256/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 8)                 200712    
                                                                 
Total params: 14915400 (56.90 MB)
Trainable params: 200712 (784.03 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
# 建立 VGG19 模型
VGG19_model = tf.keras.applications.VGG19(input_shape=input_shapes, include_top=False, weights='imagenet')
# 凍結 VGG19 的權重
VGG19_model.trainable = False

model_5 = models.Sequential(VGG19_model)
model_5.add(layers.Flatten())
model_5.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_5.summary()
# 編譯模型
model_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

80134624/80134624 [==============================] - 4s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_1 (Dense)             (None, 8)                 200712    
                                                                 
Total params: 20225096 (77.15 MB)
Trainable params: 200712 (784.03 KB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________


In [ ]:
# 建立 DenseNet121 模型
DN121_model = tf.keras.applications.DenseNet121(input_shape=input_shapes, include_top=False, weights='imagenet')
# 凍結 DenseNet121 的權重
DN121_model.trainable = False

model_6 = models.Sequential(DN121_model)
model_6.add(layers.GlobalAveragePooling2D())
model_6.add(layers.Dropout(0.5))
model_6.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_6.summary()
# 編譯模型
model_6.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

29084464/29084464 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 8)                 8200      
                                                                 
Total params: 7045704 (26.88 MB)
Trainable params: 8200 (32.03 KB)
Non-trainable params: 7037504 (26.85 MB)
_________________________________________________________________


# 訓練模型

In [ ]:
# 使用TFhub中 MobileNetV2 model
history_4 = model_4.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data)

110/110 [==============================] - 35s 240ms/step - loss: 0.7509 - accuracy: 0.7429 - val_loss: 0.7375 - val_accuracy: 0.7453


In [ ]:
# 使用 VGG19
history_5 = model_5.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data)

110/110 [==============================] - 20s 177ms/step - loss: 0.7450 - accuracy: 0.7386 - val_loss: 0.8512 - val_accuracy: 0.7213


In [ ]:
# 使用 DenseNet121
history_6 = model_6.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data)

110/110 [==============================] - 36s 227ms/step - loss: 1.2746 - accuracy: 0.5517 - val_loss: 0.6968 - val_accuracy: 0.8107


# 評估模型

In [ ]:
model_list = [model_4, model_5, model_6]
result = pd.DataFrame(columns=['Accuracy', 'Loss'])

for model in model_list:
  test_loss, test_acc = model.evaluate(test_data)
  result = pd.concat([result, pd.DataFrame({'Accuracy': [test_acc], 'Loss': [test_loss]})], ignore_index=True)

24/24 [==============================] - 2s 83ms/step - loss: 0.6952 - accuracy: 0.7947


In [ ]:
result.index = ['VGG16', 'VGG19', 'DenseNet121']

result

,Accuracy,Loss
VGG16,0.752000,0.692894
VGG19,0.710667,0.811481
DenseNet121,0.794667,0.695224
